In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import chardet
import os
import time
import ast

# ------------------ Helpers ------------------

def detect_encoding(filepath):
    with open(filepath, 'rb') as f:
        raw = f.read(10000)
    result = chardet.detect(raw)
    return result['encoding'] or 'utf-8'

def load_csv(filepath):
    try:
        encoding = detect_encoding(filepath)
        print(f"🔍 Detected encoding: {encoding}")
        return pd.read_csv(filepath, encoding=encoding).dropna(how='all')
    except UnicodeDecodeError:
        print("⚠️ Encoding issue detected. Retrying with UTF-8 and fallback open().")
        with open(filepath, mode='r', encoding='utf-8', errors='replace') as f:
            return pd.read_csv(f).dropna(how='all')

def detect_unpackable_columns(df, sample_size=5):
    unpackable = []
    for col in df.columns:
        samples = df[col].dropna().astype(str).head(sample_size)
        for val in samples:
            try:
                parsed = ast.literal_eval(val.strip())
                if isinstance(parsed, dict) and all(isinstance(v, dict) for v in parsed.values()):
                    unpackable.append(col)
                    break
            except:
                continue
    return unpackable

def unpack_column(df, col, prefix):
    rows = []
    for _, row in df.iterrows():
        raw = row[col]
        try:
            nested = ast.literal_eval(str(raw).strip())
        except:
            continue
        for subkey, subvals in nested.items():
            if not isinstance(subvals, dict):
                continue
            new_row = row.to_dict()
            new_row[f"{prefix}_key"] = subkey
            for k, v in subvals.items():
                new_row[f"{prefix}_{k}"] = v
            rows.append(new_row)
    return pd.DataFrame(rows)

# ------------------ Summary ------------------

def summarize_selected_columns(df, group_label=None):
    # Columns to summarize
    cols_to_summarize = ["Goals Home", "Away Goals", "home_possessions", "away_possessions", "home_chances", "away_chances"]
    cols_present = [c for c in cols_to_summarize if c in df.columns]

    summary_data = {"Group": group_label or "Overall"}

    for col in cols_present:
        series = pd.to_numeric(df[col], errors="coerce").dropna()
        if not series.empty:
            summary_data[f"{col} Mean"] = round(series.mean(), 2)
            summary_data[f"{col} StdDev"] = round(series.std(), 2)
            summary_data[f"{col} Min"] = round(series.min(), 2)
            summary_data[f"{col} Max"] = round(series.max(), 2)
        else:
            summary_data[f"{col} Mean"] = "NA"
            summary_data[f"{col} StdDev"] = "NA"
            summary_data[f"{col} Min"] = "NA"
            summary_data[f"{col} Max"] = "NA"

    return summary_data

# ------------------ Plots ------------------

def plot_columns(df, numeric_cols, categorical_cols, prefix):
    os.makedirs("Task_06_plots_grouped_by_stadium", exist_ok=True)
    
    selected_numeric_cols = ["Goals Home", "Away Goals", "home_possessions", "away_possessions", "home_chances", "away_chances"]
    
    for col in selected_numeric_cols:
        if col in df.columns:
            try:
                df[col].value_counts().head(10).plot(kind='bar', title=f"Top Values: {col}")
                plt.xlabel(col)
                plt.ylabel("Count")
                plt.xticks(rotation=45)
                plt.tight_layout()
                plt.savefig(f"Task_06_plots_grouped_by_stadium/{prefix}_bar_{col.replace(' ', '_')}.png")
                plt.close()
            except Exception as e:
                print(f"⚠ Bar chart failed for {col}: {e}")
        else:
            print(f"⚠ Column '{col}' not found in dataset. Skipping plot.")
            
'''
    for col in selected_numeric_cols:
        if col in df.columns:
            try:
                df[col].dropna().astype(float).plot(kind='hist', bins=20, title=f"Histogram: {col}")
                plt.xlabel(col)
                plt.ylabel("Frequency")
                plt.tight_layout()
                plt.savefig(f"Task_05_plots_without_grouping/{prefix}hist{col}.png")
                plt.close()
            except Exception as e:
                print(f"⚠ Histogram failed for {col}: {e}")
                
'''

# ------------------ Main ------------------

if __name__ == "__main__":
    print("📊 PANDAS DESCRIPTIVE STATISTICS")
    filepath = input("Enter CSV path : ").strip()
    dataset_name = os.path.splitext(os.path.basename(filepath))[0]
    group_input = input("Enter group keys (comma-separated or leave blank): ").strip()
    group_keys = [g.strip() for g in group_input.split(",") if g.strip()]

    print("📥 Loading data...")
    df = load_csv(filepath)  # Make sure load_csv doesn't limit to .head(10) for full dataset

    print("🔍 Unpacking nested columns (if any)...")
    unpack_cols = detect_unpackable_columns(df)
    for col in unpack_cols:
        df = unpack_column(df, col, col.split("_")[0])
        df.drop(columns=[col], inplace=True)

    print("📊 Summarizing...")
    start = time.perf_counter()

    # Summary list for DataFrame
    summaries = []
    summaries.append(summarize_selected_columns(df, group_label="Overall"))

    if group_keys:
        grouped = df.groupby(group_keys)
        for group_val, group_df in grouped:
            label = ", ".join(f"{k}={v}" for k, v in zip(group_keys, group_val)) if isinstance(group_val, tuple) else f"{group_keys[0]}={group_val}"
            summaries.append(summarize_selected_columns(group_df, group_label=label))

    os.makedirs("Task_06_summaries", exist_ok=True)
    out_file = f"Task_06_summaries/summary_grouped_by_stadium_{dataset_name}.csv"
    pd.DataFrame(summaries).to_csv(out_file, index=False)
    print(f"✅ Summary saved to {out_file}")
        
    # Optional plotting
    if input("📈 Generate plots? (y/n): ").strip().lower() == 'y':
        num_cols = [col for col in df.columns if pd.api.types.is_numeric_dtype(df[col])]
        cat_cols = [col for col in df.columns if col not in num_cols]
        plot_columns(df, num_cols, cat_cols, dataset_name)
        print("✅ Plots saved to Task_05_plots_without_grouping/")

    print(f"⏱ Done in {time.perf_counter() - start:.2f} seconds")

📊 PANDAS DESCRIPTIVE STATISTICS


Enter CSV path :  Premier_League.csv
Enter group keys (comma-separated or leave blank):  stadium


C:\Users\yadav\AppData\Local\Temp\ipykernel_80976\3154580382.py:146: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for group_val, group_df in grouped:


📥 Loading data...
🔍 Detected encoding: ascii
🔍 Unpacking nested columns (if any)...
📊 Summarizing...
✅ Summary saved to Task_06_summaries/summary_grouped_by_stadium_Premier_League.csv


📈 Generate plots? (y/n):  y


✅ Plots saved to Task_05_plots_without_grouping/
⏱ Done in 3.22 seconds


In [8]:
# Q1: Matches per stadium
matches_per_stadium = df.groupby("stadium").size().reset_index(name="match_count")
matches_per_stadium.sort_values(by="match_count", ascending=False)


,stadium,match_count
0,Amex Stadium,19
9,Molineux,19
19,Villa Park,19
18,Tottenham Hotspur Stadium,19
17,The King Power Stadium,19
16,The City Ground,19
15,Stamford Bridge,19
13,"St James' Park, Newcastle",19
12,Selhurst Park,19
20,Vitality Stadium,19


In [9]:
# Q5: Stadiums with highest variability in home goals
variability_home_goals = df.groupby("stadium")["Goals Home"].std().reset_index(name="std_dev_home_goals")
variability_home_goals.sort_values(by="std_dev_home_goals", ascending=False).head(5)


,stadium,std_dev_home_goals
1,Anfield,2.278774
13,"St James' Park, Newcastle",1.822536
0,Amex Stadium,1.809656
10,Nan,1.516575
4,Emirates Stadium,1.474937


In [10]:
# Q6: Stadium with highest single-match total goals
df["total_goals"] = df["Goals Home"] + df["Away Goals"]
max_total_row = df.loc[df["total_goals"].idxmax()]
max_total_row[["stadium", "Goals Home", "Away Goals", "total_goals"]]


stadium        Etihad Stadium
Goals Home                  6
Away Goals                  3
total_goals                 9
Name: 304, dtype: object

In [11]:
# Q7: Stadiums with lowest average total goals
df["total_goals"] = df["Goals Home"] + df["Away Goals"]
low_scoring_stadiums = df.groupby("stadium")["total_goals"].mean().reset_index(name="avg_total_goals")
low_scoring_stadiums.sort_values(by="avg_total_goals", ascending=True).head(5)


,stadium,avg_total_goals
15,Stamford Bridge,2.052632
9,Molineux,2.052632
6,Goodison Park,2.111111
12,Selhurst Park,2.315789
11,Old Trafford,2.388889


In [12]:
# Q8: Stadium with strongest home advantage
home_avg = df.groupby("stadium")["Goals Home"].mean()
away_avg = df.groupby("stadium")["Away Goals"].mean()
advantage_df = (home_avg - away_avg).reset_index(name="home_goal_advantage")
advantage_df.sort_values(by="home_goal_advantage", ascending=False).head(5)


,stadium,home_goal_advantage
5,Etihad Stadium,2.263158
1,Anfield,1.555556
4,Emirates Stadium,1.473684
11,Old Trafford,1.388889
13,"St James' Park, Newcastle",1.157895


In [13]:
# Q10: Correlation between possession and home goals per stadium
correlation_results = {}
for stadium, group in df.groupby("stadium"):
    if group["home_possessions"].notna().sum() > 1 and group["Goals Home"].notna().sum() > 1:
        corr = group["home_possessions"].corr(group["Goals Home"])
        correlation_results[stadium] = corr

# Sort by strongest correlation
sorted_corr = sorted(correlation_results.items(), key=lambda x: x[1] if x[1] is not None else -999, reverse=True)
sorted_corr[:5]  # Top 5 stadiums


[('Selhurst Park', 0.44555488287900286),
 ('Anfield', 0.31790190932329687),
 ('Elland Road', 0.17138578044066882),
 ('London Stadium', 0.15208980714347634),
 ('Goodison Park', 0.15043113897385257)]